In [120]:
import gffutils
import subprocess
import pandas as pd
from Bio import SeqIO
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import pandas as pd
import pickle
import os
from BCBio import GFF
from collections import defaultdict
from pybedtools import BedTool

In [57]:
#Задача - получить для каждой lncRNA(гена) .gff файл, содержащий все транскрипты(процессированные)(и экзоны) этого гена
#Потом с помощью gffread получим для каждого такого gff файла fa файл с последовательностями процессированных транскриптов
#gffread gencode.v25.annotation.gff3 -g hg38.fa -w hg38.ALL.fna

In [74]:
lncRNAMatrix = pd.read_csv("../H3K27me3/lncRNA_matrix_filtered.csv", sep="\t", index_col=0)

In [90]:
peaks = pd.read_csv("../H3K27me3/bw/subsampleHimericBam_peaks.bed", sep="\t", skiprows=1, header=None)

In [92]:
peaks = peaks[[0, 1, 2, 3]]

In [93]:
peaks.head()

,0,1,2,3
0,chr1,801525,802577,subsampleHimericBam_peaks.bed_broadRegion2
1,chr1,804282,806294,subsampleHimericBam_peaks.bed_broadRegion3
2,chr1,807213,810961,subsampleHimericBam_peaks.bed_broadRegion4
3,chr1,814722,826652,subsampleHimericBam_peaks.bed_broadRegion5
4,chr1,828257,828984,subsampleHimericBam_peaks.bed_broadRegion6


In [110]:
peaksDict = {}
for i, row in peaks.iterrows():
    peaksDict[row[3]] = (row[0], row[1], row[2])

In [75]:
#hg19 айдишники
lncRNAGenes = list(lncRNAMatrix.index)

In [76]:
with open("../H3K27me3/lncRNA_Peaks_Correlations_corrected.pickle", 'rb') as f:
    corr = pickle.load(f)

In [77]:
in_file = "../H3K27me3/gencode.v29lift37.long_noncoding_RNAs.gff3"
in_handle = open(in_file)

limit_info = dict(
    gff_id = ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 
 'chr19', 'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9'],
    gff_type = ["gene"])

lncRNAgenes = []
for rec in GFF.parse(in_handle, limit_info=limit_info):
    lncRNAgenes.append((rec.id, rec.features)) 
    
in_handle.close()

In [62]:
tmp = [k.split('.')[0] for k in lncRNAGenes]

In [78]:
#38 айдишники
lncRNAgenesIds = [g.id for i, j in lncRNAgenes for g in j if g.id.split('.')[0] in tmp]

In [79]:
lncRNAgenesIds.sort(key = lambda x: x.split('.')[0])
lncRNAGenes.sort(key = lambda x: x.split('.')[0])

In [80]:
#в gff лежат названия айдишников для версии ghr38, а в данных hg19
convertIds = {i:j for i, j in zip(lncRNAGenes, lncRNAgenesIds)}

In [ ]:
#Делается один раз
db = gffutils.create_db("../H3K27me3/gencode.v29lift37.long_noncoding_RNAs.gff3", dbfn='../H3K27me3/long_noncoding_RNAs.db')

In [81]:
db = gffutils.FeatureDB('../H3K27me3/long_noncoding_RNAs.db', keep_order=True)

In [83]:
#все транскрипты в одном файле
for g in lncRNAGenes:
    gene = db[convertIds[g]]

    with open("../H3K27me3/lnc_gff/all_transcripts_lncRNA.gff3", "a") as myfile:
        for i in db.children(gene, featuretype='transcript', order_by='start'):
            myfile.write(str(i) + "\n")
        for i in db.children(gene, featuretype='exon', order_by='start'):
            myfile.write(str(i) + "\n")
            
cmd = 'gffread ../H3K27me3/lnc_gff/all_transcripts_lncRNA.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/all_transcripts_lncRNA.fa'     
print(cmd)
p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
p.wait()

gffread ../H3K27me3/lnc_gff/all_transcripts_lncRNA.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/all_transcripts_lncRNA.fa


0

In [84]:
#свой файл для каждой lncRNA
for g in lncRNAGenes:
    gene = db[convertIds[g]]

    with open("../H3K27me3/lnc_gff/" + g + ".gff3", "a") as myfile:
        for i in db.children(gene, featuretype='transcript', order_by='start'):
            myfile.write(str(i) + "\n")
        for i in db.children(gene, featuretype='exon', order_by='start'):
            myfile.write(str(i) + "\n")
            
    cmd = 'gffread ../H3K27me3/lnc_gff/' + g + '.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/' + g + '_transcript_lncRNA.fa'     
    print(cmd)
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    p.wait()

gffread ../H3K27me3/lnc_gff/ENSG00000093100.13.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000093100.13_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000103472.5.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000103472.5_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000116652.5.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000116652.5_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000116883.8.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000116883.8_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000117242.7.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000117242.7_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000120664.6.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000120664.6_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000124835.2.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000124835.2_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000125804.9.gff3 -g ../hg1

gffread ../H3K27me3/lnc_gff/ENSG00000183154.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000183154.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000183250.7.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000183250.7_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000184068.2.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000184068.2_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000184441.4.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000184441.4_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000185044.10.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000185044.10_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000185065.6.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000185065.6_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000185168.5.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000185168.5_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000185904.7.gff3 -g ../hg1

gffread ../H3K27me3/lnc_gff/ENSG00000204282.3.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000204282.3_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000204584.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000204584.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000204588.5.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000204588.5_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000204685.5.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000204685.5_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000204706.10.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000204706.10_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000204758.3.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000204758.3_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000204802.3.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000204802.3_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000204850.3.gff3 -g ../hg1

gffread ../H3K27me3/lnc_gff/ENSG00000215256.3.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000215256.3_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000215374.5.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000215374.5_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000215386.6.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000215386.6_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000215417.6.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000215417.6_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000215424.5.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000215424.5_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000215769.4.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000215769.4_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000215808.2.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000215808.2_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000216895.4.gff3 -g ../hg19/

gffread ../H3K27me3/lnc_gff/ENSG00000224812.2.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000224812.2_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000224818.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000224818.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000224822.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000224822.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000224843.2.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000224843.2_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000224870.3.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000224870.3_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000224885.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000224885.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000224888.3.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000224888.3_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000224897.2.gff3 -g ../hg19/

...

gffread ../H3K27me3/lnc_gff/ENSG00000271795.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000271795.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000271797.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000271797.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000271811.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000271811.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000271816.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000271816.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000271833.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000271833.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000271840.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000271840.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000271843.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000271843.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000271851.1.gff3 -g ../hg19/

gffread ../H3K27me3/lnc_gff/ENSG00000272219.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272219.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272221.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272221.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272264.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272264.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272267.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272267.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272269.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272269.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272273.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272273.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272277.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272277.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272288.1.gff3 -g ../hg19/

gffread ../H3K27me3/lnc_gff/ENSG00000272686.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272686.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272688.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272688.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272690.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272690.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272695.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272695.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272702.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272702.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272711.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272711.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272716.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000272716.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000272720.1.gff3 -g ../hg19/

gffread ../H3K27me3/lnc_gff/ENSG00000273018.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273018.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273033.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273033.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273035.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273035.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273038.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273038.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273058.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273058.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273062.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273062.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273064.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273064.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273066.1.gff3 -g ../hg19/

gffread ../H3K27me3/lnc_gff/ENSG00000273448.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273448.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273449.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273449.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273451.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273451.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273456.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273456.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273466.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273466.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273472.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273472.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273486.1.gff3 -g ../hg19/allChr.fa -w ../H3K27me3/lnc_gff/ENSG00000273486.1_transcript_lncRNA.fa
gffread ../H3K27me3/lnc_gff/ENSG00000273489.1.gff3 -g ../hg19/

In [112]:
#Для каждой lncRNA нужен список пиков, с которыми она скоррелированна
d = defaultdict(list)
for g, p, c in corr:
    d[g].append((p, peaksDict[p]))

In [117]:
#теперь надо создать bed(типа chr1 5 10) файлы с координатами, а потом с помощью bedtools getfasta получить фаста файл
for k, v in d.items():
    with open("../H3K27me3/peaks_coords/seqs_for_lnc_" + k + '.bed', 'w') as fp:
        fp.write('\n'.join('%s\t%s\t%s' % (x[1][0], x[1][1], x[1][2]) for x in v))

In [125]:
for f in [f for f in os.listdir('../H3K27me3/peaks_coords/') if f.endswith('.bed')]:
    bed = BedTool("../H3K27me3/peaks_coords/" + f)
    bed = bed.sequence(fi="../hg19/allChr.fa", fo="../H3K27me3/peaks_coords/" + os.path.splitext(f)[0] + '.fa', name=True)